In [37]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [40]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [41]:
%%capture pwd
!pwd

In [42]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
yConfig = alt.Axis(
    title="%",
    domain=False,
    ticks=False,
    labelAlign="left",
    labelBaseline="middle",
    labelPadding=-5,
    labelOffset=-10,
    titleX=35,
    titleY=-6,
    titleAngle=0,
    titleFontSize=11,
    titleAlign="left",
    tickCount=5,
    format=".0f",
)
xConfig = alt.Axis(
    grid=False,
    title="",
    titleAnchor="end",
    titleY=-15,
    labelColor=colors["eco-gray"],
    titleColor=colors["eco-gray"],
    tickColor=colors["eco-gray"],
    domainColor=colors["eco-gray"],
)
height = config["height"]
width = config["width"]
uid, height, width

('2021-06-04-what-are-the-big-economic-challenges-facing-the-new-welsh-government',
 300,
 500)

Common

In [43]:
recessions = [
    #     {"start": "1990-01-01", "end": "1993-01-01", "event": "recession"},
    #     {"start": "2008-01-01", "end": "2011-01-01", "event": "recession"},
    {"start": "2020Q1", "q3": "2020Q3", "q4": "2020Q4", "event": "covid"},
]
color_dict = {"covid": "#F3F4D8", "recession": "#E2E2E2"}
dr = pd.DataFrame(recessions)
dr["color"] = [color_dict[i] for i in dr["event"]]

In [44]:
recessions = [
    #     {"start": "1990-01-01", "end": "1993-01-01", "event": "recession"},
    #     {"start": "2008-01-01", "end": "2011-01-01", "event": "recession"},
    {"start": "2020-03-01", "q3": "2020-09-01", "q4": "2021-04-01", "event": "covid"},
]
color_dict = {"covid": "#F3F4D8", "recession": "#E2E2E2"}
dr2 = pd.DataFrame(recessions)
dr2["color"] = [color_dict[i] for i in dr["event"]]

# Fig 1

In [45]:
df = pd.read_excel(
    "raw/Data for graphs (Rodriguez & Ifan)  ECO3.xlsx", sheet_name="Figure 1"
).dropna(how="all")
df.columns = ["date", "region", "value"]

In [46]:
f = "fig1_gdp"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
df.head()

,date,region,value
0,2018Q3,UK,0.7
1,2018Q3,England,0.8
2,2018Q3,Wales,0.2
3,2018Q3,Scotland,0.1
4,2018Q3,Northern Ireland,0.0


In [47]:
bars = alt.Chart(f1).encode(
    y=alt.Y(
        "value:Q",
        stack=False,
        title="",
        axis=alt.Axis(
            grid=True,
            title="%",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=20,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=7,
            format=".0f",
        ),
        scale=alt.Scale(domain=[-22, 15]),
    ),
    x=alt.X(
        "date:N",
        title="",
        axis=alt.Axis(
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            orient="bottom",
            labelExpr="indexof(datum.value,'Q1')>-1?slice(datum.value,0,4)+' '+slice(datum.value,4):slice(datum.value,4)",
            labelAngle=0,
        ),
    ),
)

xAxis = (
    alt.Chart(pd.DataFrame([{"value": 0}]))
    .mark_rule(size=0.5)
    .encode(
        y=alt.Y("value:Q"),
    )
)
rect = (
    alt.Chart(dr)
    .mark_rect(blend="darken", xOffset=-22, x2Offset=22)
    .encode(
        x2="q3:N",
        color=alt.Color("color:N", scale=None),
        x=alt.X(
            "start:N",
            title="",
            axis=alt.Axis(
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                orient="bottom",
                labelExpr="indexof(datum.value,'Q1')>-1?slice(datum.value,0,4)+' '+slice(datum.value,4):slice(datum.value,4)",
                labelAngle=0,
            ),
        ),
    )
)
bars1 = bars.mark_bar(
    size=8, xOffset=-12, color=colors["eco-purple"], opacity=0.8
).transform_filter("datum.region=='England'")
bars2 = bars.mark_bar(
    size=8, xOffset=-4, color=colors["eco-pink"], opacity=0.8
).transform_filter("datum.region=='Wales'")
bars3 = bars.mark_bar(
    size=8, xOffset=4, color=colors["eco-mid-blue"], opacity=0.8
).transform_filter("datum.region=='Scotland'")
bars4 = bars.mark_bar(
    size=8, xOffset=12, color=colors["eco-green"], opacity=0.8
).transform_filter("datum.region=='Northern Ireland'")
line = bars.mark_line(color=colors["eco-blue"], opacity=0.8).transform_filter(
    "datum.region=='UK'"
)

text = line.mark_text(
    yOffset=15, xOffset=-5, size=10, align="right", color=colors["eco-blue"]
).encode(text=alt.Text("value:Q", format=".1f"))
label1 = (
    alt.Chart(
        pd.DataFrame(
            [{"x": "2019Q3", "y": -2.5, "t": "England", "c": colors["eco-purple"]}]
        )
    )
    .mark_text(xOffset=-17, align="right", angle=270, size=10, baseline="middle")
    .encode(text="t", x="x:N", y="y:Q", color=alt.Color("c:N", scale=None))
)
label2 = (
    alt.Chart(
        pd.DataFrame(
            [{"x": "2019Q3", "y": -2.5, "t": "Wales", "c": colors["eco-pink"]}]
        )
    )
    .mark_text(xOffset=-5, align="right", angle=270, size=10, baseline="middle")
    .encode(text="t", x="x:N", y="y:Q", color=alt.Color("c:N", scale=None))
)
label3 = (
    alt.Chart(
        pd.DataFrame(
            [{"x": "2019Q3", "y": -2.5, "t": "Scotland", "c": colors["eco-mid-blue"]}]
        )
    )
    .mark_text(xOffset=5, align="right", angle=270, size=10, baseline="middle")
    .encode(text="t", x="x:N", y="y:Q", color=alt.Color("c:N", scale=None))
)
label4 = (
    alt.Chart(
        pd.DataFrame(
            [
                {
                    "x": "2019Q3",
                    "y": -2.5,
                    "t": "Northern Ireland",
                    "c": colors["eco-green"],
                }
            ]
        )
    )
    .mark_text(xOffset=15, align="right", angle=270, size=10, baseline="middle")
    .encode(text="t", x="x:N", y="y:Q", color=alt.Color("c:N", scale=None))
)
label5 = (
    alt.Chart(
        pd.DataFrame([{"x": "2020Q1", "y": -7.5, "t": "UK", "c": colors["eco-blue"]}])
    )
    .mark_text(xOffset=7, align="right", angle=0, size=10, baseline="middle")
    .encode(text="t", x="x:N", y="y:Q", color=alt.Color("c:N", scale=None))
)
label6 = (
    alt.Chart(
        pd.DataFrame([{"x": "2020Q1", "y": 12.2, "t": "➡ COVID-19", "c": "#c1a62c"}])
    )
    .mark_text(xOffset=-22, align="left", angle=0, size=10, baseline="middle")
    .encode(text="t", x="x:N", y="y:Q", color=alt.Color("c:N", scale=None))
)
labels = label1 + label2 + label3 + label4 + label5 + label6

layer = (
    (xAxis + rect + bars1 + bars2 + bars3 + bars4 + line + text + labels)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)

# Fig 2

In [48]:
df = (
    pd.read_excel(
        "raw/Data for graphs (Rodriguez & Ifan)  ECO3.xlsx", sheet_name="Figure 2"
    )
    .dropna(how="all")
    .drop("date", axis=1)
    .set_index(["date1", "index_n"])
    .stack()
    .reset_index()
)
df.columns = ["date", "index", "region", "value"]
df["date"] = df["date"].str.replace(" ", "")
df["index"] = df["index"].str.replace("Index of ", "")

In [49]:
f = "fig2_output_indicators"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
df.head()

,date,index,region,value
0,2018Q3,Construction,UK,100.9
1,2018Q3,Construction,Wales,105.4
2,2018Q3,Market Services,UK,100.4
3,2018Q3,Market Services,Wales,100.5
4,2018Q3,Production,UK,100.4


In [50]:
base = (
    alt.Chart(f2).encode(
        x=alt.X(
            "date:N",
            title="",
            axis=alt.Axis(
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                orient="bottom",
                labelExpr="indexof(datum.value,'Q1')>-1?slice(datum.value,0,4)+' '+slice(datum.value,4):slice(datum.value,4)",
                labelAngle=0,
            ),
        ),
        y=alt.Y(
            "value:Q",
            title="",
            axis=alt.Axis(
                title="Index (2018=100)",
                domain=False,
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=25,
                titleY=-5,
                titleAngle=0,
                titleFontSize=10,
                titleAlign="left",
                tickCount=5,
                format=".0f",
            ),
            scale=alt.Scale(domain=[60, 110]),
        ),
        color=alt.Color(
            "index:N",
            scale=alt.Scale(
                range=[
                    colors["eco-turquiose"],
                    colors["eco-light-blue"],
                    colors["eco-blue"],
                ]
            ),
            legend=None,
        ),
    )
).transform_filter("datum.region=='Wales'")
line = base.mark_line()
text = (
    base.mark_text(align="left", xOffset=5, yOffset=-4)
    .encode(text="index:N")
    .transform_filter("datum.date=='2020Q4'")
).transform_filter("datum.index!='Production'")
text2 = (
    base.mark_text(align="left", xOffset=5, yOffset=10)
    .encode(text="index:N")
    .transform_filter("datum.date=='2020Q4'")
).transform_filter("datum.index=='Production'")
xAxis = (
    alt.Chart(pd.DataFrame([{"value": 100}]))
    .mark_rule(size=0.5)
    .encode(
        y=alt.Y("value:Q", scale=alt.Scale(domain=[60, 110])),
    )
)
rect = (
    alt.Chart(dr)
    .mark_rect(blend="darken", x2Offset=20)
    .encode(
        x=alt.X(
            "start:N",
            title="",
            axis=alt.Axis(
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                orient="bottom",
                labelExpr="indexof(datum.value,'Q1')>-1?slice(datum.value,0,4)+' '+slice(datum.value,4):slice(datum.value,4)",
                labelAngle=0,
            ),
        ),
        x2="q4:N",
        color=alt.Color("color:N", scale=None),
    )
)
label6 = (
    alt.Chart(
        pd.DataFrame([{"x": "2020Q1", "y": 64, "t": "➡ COVID-19", "c": "#c1a62c"}])
    )
    .mark_text(xOffset=0, align="left", angle=0, size=10, baseline="middle")
    .encode(text="t", x="x:N", y="y:Q", color=alt.Color("c:N", scale=None))
)
layer = (
    (xAxis + rect + line + text + text2 + label6)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)

# Fig 3

## a

In [51]:
df = pd.read_excel(
    "raw/Data for graphs (Rodriguez & Ifan)  ECO3.xlsx",
    sheet_name="Figure 3",
    skiprows=1,
    usecols="A:E",
).dropna(how="all")
df = df.set_index("Date").stack().reset_index()
df.columns = ["date", "region", "value"]

In [52]:
f = "fig3a_job_adverts"
f3a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3a = df
df.head()

,date,region,value
0,2019-01-04,England,95.0
1,2019-01-04,Wales,75.0
2,2019-01-04,Scotland,80.8
3,2019-01-04,Northern Ireland,134.1
4,2019-01-11,England,104.5


In [53]:
base = (
    alt.Chart(f3a)
    .encode(
        x=alt.X(
            "date:T",
            title="",
            axis=alt.Axis(
                grid=False,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        y=alt.Y(
            "value:Q",
            title="",
            axis=alt.Axis(
                title="Index (February 2020 average=100)",
                domain=False,
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=5,
                titleY=-5,
                titleAngle=0,
                titleFontSize=10,
                titleAlign="left",
                tickCount=5,
                format=".0f",
            ),
            scale=alt.Scale(domain=[30, 130]),
        ),
        color=alt.Color(
            "region:N",
            scale=alt.Scale(
                range=[
                    colors["eco-purple"],
                    colors["eco-mid-blue"],
                    colors["eco-pink"],
                ]
            ),
            legend=None,
        ),
    )
    .transform_filter("datum.region!='Northern Ireland'")
)
line = base.mark_line()
text = (
    (base.mark_text(align="left", xOffset=5, yOffset=4).encode(text="region:N"))
    .transform_filter("datum.date>toDate('2021-04-08')")
    .transform_filter("datum.region=='Wales'")
)
text2 = (
    (base.mark_text(align="left", xOffset=5, yOffset=-4).encode(text="region:N"))
    .transform_filter("datum.date>toDate('2021-04-08')")
    .transform_filter("datum.region=='Scotland'")
)
text3 = (
    (base.mark_text(align="left", xOffset=5, yOffset=0).encode(text="region:N"))
    .transform_filter("datum.date>toDate('2021-04-08')")
    .transform_filter("datum.region=='England'")
)
xAxis = (
    alt.Chart(pd.DataFrame([{"value": 100}]))
    .mark_rule(size=0.5)
    .encode(
        y=alt.Y("value:Q", scale=alt.Scale(domain=[30, 130])),
    )
)
rect = (
    alt.Chart(dr2)
    .mark_rect(blend="darken", x2Offset=5)
    .encode(
        x=alt.X("start:T"),
        x2="q4:T",
        color=alt.Color("color:N", scale=None),
    )
)
label6 = (
    alt.Chart(
        pd.DataFrame([{"x": "2020-03-01", "y": 114, "t": "➡ COVID-19", "c": "#c1a62c"}])
    )
    .mark_text(xOffset=0, align="left", angle=0, size=10, baseline="middle")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
layer = (
    (xAxis + rect + line + label6 + text + text2 + text3)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)

## b

In [54]:
df = pd.read_excel(
    "raw/Data for graphs (Rodriguez & Ifan)  ECO3.xlsx",
    sheet_name="Figure 3",
    skiprows=1,
    usecols="H:J",
).dropna(how="all")
df = df.set_index("date").stack().reset_index()
df.columns = ["date", "region", "value"]
df["region"] = df["region"].str.replace(".1", "")

<ipython-input-54-0b5cdaaf925c>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df["region"] = df["region"].str.replace(".1", "")


In [55]:
f = "fig3b_job_vacancies"
f3b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3b = df
df.head()

,date,region,value
0,2018-12-30,UK,853.00
1,2018-12-30,Wales,41.31
2,2019-01-30,UK,851.00
3,2019-01-30,Wales,42.63
4,2019-02-28,UK,842.00


In [56]:
base = (
    alt.Chart(f3b)
    .encode(
        x=alt.X(
            "date:T",
            title="",
            axis=alt.Axis(
                grid=False,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        y=alt.Y(
            "value:Q",
            title="",
            axis=alt.Axis(
                title="Number of vacancies (thousands)",
                domain=False,
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=5,
                titleY=-5,
                titleAngle=0,
                titleFontSize=10,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                tickCount=5,
                format=".0f",
            ),
            scale=alt.Scale(domain=[300, 900]),
        ),
        color=alt.Color(
            "region:N",
            scale=alt.Scale(
                range=[
                    colors["eco-gray"],
                    colors["eco-pink"],
                ]
            ),
            legend=None,
        ),
    )
    .transform_filter("datum.region!='Northern Ireland'")
)
line1 = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "value:Q",
            title="",
            axis=alt.Axis(
                title="",
                domain=False,
                ticks=False,
                grid=True,
                labelAlign="right",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                labelColor=colors["eco-pink"],
                titleX=5,
                titleY=-5,
                titleAngle=0,
                titleFontSize=10,
                titleAlign="right",
                tickCount=5,
                format=".0f",
            ),
            scale=alt.Scale(domain=[0, 50]),
        )
    )
    .transform_filter("datum.region=='Wales'")
)
line2 = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "value:Q",
            title="",
            axis=alt.Axis(
                title="Number of vacancies (thousands)",
                domain=False,
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=30,
                titleY=-5,
                titleAngle=0,
                titleFontSize=10,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                tickCount=5,
                format=".0f",
            ),
            scale=alt.Scale(domain=[300, 900]),
        )
    )
    .transform_filter("datum.region=='UK'")
)
rect = (
    alt.Chart(dr2)
    .mark_rect(blend="darken", x2Offset=5)
    .encode(
        x=alt.X("start:T"),
        x2="q4:T",
        color=alt.Color("color:N", scale=None),
    )
)
label6 = (
    alt.Chart(
        pd.DataFrame([{"x": "2020-03-01", "y": 47, "t": "➡ COVID-19", "c": "#c1a62c"}])
    )
    .mark_text(xOffset=0, align="left", angle=0, size=10, baseline="middle")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
label5 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": "2020-11-01", "y": 17, "t": "UK", "c": colors["eco-gray"]},
                {"x": "2020-08-01", "y": 24, "t": "Wales", "c": colors["eco-pink"]},
            ]
        )
    )
    .mark_text(xOffset=0, align="left", angle=0, size=10, baseline="middle")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
layer = (
    ((line2) + (rect + label6 + label5 + line1))
    .resolve_scale(y="independent")
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)

# Fig 4

In [57]:
df = pd.read_excel(
    "raw/Data for graphs (Rodriguez & Ifan)  ECO3.xlsx",
    sheet_name="Figure 4",
).dropna(how="all")
df = df.set_index("Date1").stack().reset_index()
df.columns = ["date", "region", "value"]
df["region"] = df["region"].replace("NI", "Northern Ireland")

In [58]:
f = "fig4_unemployment"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
df.head()

,date,region,value
0,2017-01-01,England,4.836335
1,2017-01-01,Northern Ireland,6.014223
2,2017-01-01,Scotland,4.702146
3,2017-01-01,Wales,4.395629
4,2017-02-01,England,4.762100


In [59]:
base = alt.Chart(f4).encode(
    x=alt.X(
        "date:T",
        title="",
        axis=alt.Axis(
            grid=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    y=alt.Y(
        "value:Q",
        title="",
        axis=alt.Axis(
            title="%",
            domain=False,
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=15,
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleAlign="left",
            tickCount=5,
            format=".0f",
        ),
        scale=alt.Scale(domain=[2, 6]),
    ),
    color=alt.Color(
        "region:N",
        scale=alt.Scale(
            range=[
                colors["eco-purple"],
                colors["eco-green"],
                colors["eco-mid-blue"],
                colors["eco-pink"],
            ]
        ),
        legend=None,
    ),
)
line = base.mark_line()
text = (
    base.mark_text(align="left", xOffset=5, yOffset=4).encode(text="region:N")
).transform_filter("datum.date>toDate('2021-01-08')")
rect = (
    alt.Chart(dr2)
    .mark_rect(blend="darken", x2Offset=5)
    .encode(
        x=alt.X("start:T"),
        x2="q4:T",
        color=alt.Color("color:N", scale=None),
    )
)
label6 = (
    alt.Chart(
        pd.DataFrame([{"x": "2020-03-01", "y": 5.7, "t": "➡ COVID-19", "c": "#c1a62c"}])
    )
    .mark_text(xOffset=0, align="left", angle=0, size=10, baseline="middle")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
layer = (
    (rect + line + label6 + text)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)

# Fig 5

In [60]:
df = pd.read_excel(
    "raw/Data for graphs (Rodriguez & Ifan)  ECO3.xlsx",
    sheet_name="Figure 5",
).dropna(how="all")

In [61]:
f = "fig5_claimant"
f5 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f5.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f5 = df
df.head()

,Month,Claimant_Count,Percent (%)
0,2017-10-01,38.497,2.496323
1,2017-11-01,38.682,2.508320
2,2017-12-01,39.261,2.545865
3,2018-01-01,39.648,2.518557
4,2018-02-01,40.439,2.568803


In [62]:
base = alt.Chart(f5).encode(
    x=alt.X(
        "Month:T",
        title="",
        axis=alt.Axis(
            grid=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    )
)
line = base.mark_line(color=colors["eco-pink"]).encode(
    y=alt.Y(
        "Percent (%):Q",
        title="",
        axis=alt.Axis(
            title="Claimant count as % of the workforce",
            domain=False,
            ticks=False,
            grid=True,
            labelAlign="right",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            labelColor=colors["eco-pink"],
            titleX=-15,
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleAlign="right",
            tickCount=5,
            titleColor=colors["eco-pink"],
            format=".0f",
        ),
        scale=alt.Scale(domain=[0, 8]),
    )
)
bars = base.mark_bar(color=colors["eco-gray"], opacity=0.8).encode(
    y=alt.Y(
        "Claimant_Count:Q",
        title="",
        axis=alt.Axis(
            title="Claimant count (thousands)",
            domain=False,
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=5,
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleAlign="left",
            titleColor=colors["eco-gray"],
            labelColor=colors["eco-gray"],
            tickCount=5,
            format=".0f",
        ),
        scale=alt.Scale(domain=[30, 130]),
    )
)
rect = (
    alt.Chart(dr2)
    .mark_rect(blend="darken", x2Offset=5)
    .encode(
        x=alt.X("start:T"),
        x2="q4:T",
        color=alt.Color("color:N", scale=None),
    )
)
label6 = (
    alt.Chart(
        pd.DataFrame([{"x": "2020-03-01", "y": 7.6, "t": "➡ COVID-19", "c": "#c1a62c"}])
    )
    .mark_text(xOffset=0, align="left", angle=0, size=10, baseline="middle")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
layer = (
    ((bars) + (rect + label6 + line))
    .resolve_scale(y="independent")
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)

# Fig 6

In [63]:
df = pd.read_excel(
    "raw/Data for graphs (Rodriguez & Ifan)  ECO3.xlsx",
    sheet_name="Figure 6",
).dropna(how="all")
df["Date"] = [
    pd.to_datetime("2020-03-01") + pd.to_timedelta(str(i) + "d") for i in range(365)
]
df = df.set_index("Date").stack().reset_index()
df.columns = ["date", "region", "value"]
df["region"] = df["region"].replace("NorthernIreland", "Northern Ireland")

In [64]:
f = "fig6_furlough"
f6 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f6.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f6 = df
df.head()

,date,region,value
0,2020-03-01,England,0.061190
1,2020-03-01,Northern Ireland,0.057901
2,2020-03-01,Scotland,0.066865
3,2020-03-01,Wales,0.061225
4,2020-03-02,England,0.073780


In [65]:
base = alt.Chart(f6).encode(
    x=alt.X(
        "date:T",
        title="",
        axis=alt.Axis(
            grid=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    y=alt.Y(
        "value:Q",
        title="",
        axis=alt.Axis(
            title="%",
            domain=False,
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=20,
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleAlign="left",
            tickCount=5,
            format=".0f",
        ),
        scale=alt.Scale(domain=[0, 30]),
    ),
    color=alt.Color(
        "region:N",
        scale=alt.Scale(
            range=[
                colors["eco-purple"],
                colors["eco-green"],
                colors["eco-mid-blue"],
                colors["eco-pink"],
            ]
        ),
        legend=None,
    ),
)
line = base.mark_line()
text1 = (
    (base.mark_text(align="left", xOffset=5, yOffset=5).encode(text="region:N"))
    .transform_filter("datum.date>toDate('2021-02-27')")
    .transform_filter("datum.region=='Wales'")
)
text2 = (
    (base.mark_text(align="left", xOffset=5, yOffset=-13).encode(text="region:N"))
    .transform_filter("datum.date>toDate('2021-02-27')")
    .transform_filter("datum.region=='England'")
)
text3 = (
    (base.mark_text(align="left", xOffset=5, yOffset=0).encode(text="region:N"))
    .transform_filter("datum.date>toDate('2021-02-27')")
    .transform_filter("datum.region=='Scotland'")
)
text4 = (
    (base.mark_text(align="left", xOffset=5, yOffset=0).encode(text="region:N"))
    .transform_filter("datum.date>toDate('2021-02-27')")
    .transform_filter("datum.region=='Northern Ireland'")
)
rect = (
    alt.Chart(dr2)
    .mark_rect(blend="darken", x2Offset=5)
    .encode(
        x=alt.X("start:T"),
        x2="q4:T",
        color=alt.Color("color:N", scale=None),
    )
)
label6 = (
    alt.Chart(
        pd.DataFrame([{"x": "2020-03-01", "y": 3.3, "t": "➡ COVID-19", "c": "#c1a62c"}])
    )
    .mark_text(xOffset=0, align="left", angle=0, size=10, baseline="middle")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
layer = (
    (rect + line + label6 + text1 + text2 + text3 + text4)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)

# Fig 7

In [72]:
df1 = pd.read_excel(
    "raw/Data for graphs (Rodriguez & Ifan)  ECO3.xlsx",
    sheet_name="Figure 7",
    usecols="A:B",
    skiprows=1,
).dropna(how="all")
df1.columns = ["category", "value"]
df1["figure"] = "Earnings decile"
df2 = pd.read_excel(
    "raw/Data for graphs (Rodriguez & Ifan)  ECO3.xlsx",
    sheet_name="Figure 7",
    usecols="D:E",
    skiprows=1,
).dropna(how="all")
df2.columns = ["category", "value"]
df2["figure"] = "Age"
df3 = pd.read_excel(
    "raw/Data for graphs (Rodriguez & Ifan)  ECO3.xlsx",
    sheet_name="Figure 7",
    usecols="G:H",
    skiprows=1,
).dropna(how="all")
df3.columns = ["category", "value"]
df3["figure"] = "Gender"
df4 = pd.read_excel(
    "raw/Data for graphs (Rodriguez & Ifan)  ECO3.xlsx",
    sheet_name="Figure 7",
    usecols="J:K",
    skiprows=1,
).dropna(how="all")
df4.columns = ["category", "value"]
df4["figure"] = "Ethnicity"

df = pd.concat([df1, df2, df3, df4])

In [73]:
f = "fig7_lockdown"
f7 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f7.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f7 = df
df.head()

,category,value,figure
0,lowest,50.000000,Earnings decile
1,2,24.000000,Earnings decile
2,3,22.500000,Earnings decile
3,4,14.634150,Earnings decile
4,5,8.888889,Earnings decile


In [74]:
base = (
    alt.Chart(f7)
    .mark_bar(size=12)
    .encode(
        x=alt.X(
            "category:N",
            title="",
            axis=alt.Axis(
                grid=False,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                labelAngle=-55,
            ),
            sort=None,
        ),
        y=alt.Y("value:Q", title="", axis=None),
    )
)
step = 14
height = 200
bars1 = (
    base.mark_bar(color=colors["eco-turquiose"], opacity=0.8)
    .transform_filter("datum.figure=='Earnings decile'")
    .properties(
        height=height,
        width=alt.Step(step),
    )
)
text1 = bars1.mark_text(color=colors["eco-turquiose"], yOffset=-7, xOffset=0).encode(
    text=alt.Text("value:N", format=".0f")
)
text1b = (
    text1.mark_text(color=colors["eco-turquiose"], angle=270, align="left", yOffset=-3)
    .encode(text="figure:N", x="x:N", y="y:Q")
    .transform_calculate(x="' '", y="0")
    .transform_filter("datum.value<3")
)
bars2 = (
    base.mark_bar(color=colors["eco-light-blue"], opacity=0.8)
    .transform_filter("datum.figure=='Age'")
    .properties(
        height=height,
        width=alt.Step(step),
    )
)
text2 = bars2.mark_text(color=colors["eco-light-blue"], yOffset=-7, xOffset=0).encode(
    text=alt.Text("value:N", format=".0f")
)
text2b = (
    text2.mark_text(color=colors["eco-light-blue"], angle=270, align="left", yOffset=-3)
    .encode(text="figure:N", x="x:N", y="y:Q")
    .transform_calculate(x="' '", y="0")
    .transform_filter("datum.value<11")
)
bars3 = (
    base.mark_bar(color=colors["eco-mid-blue"], opacity=0.8)
    .transform_filter("datum.figure=='Gender'")
    .properties(
        height=height,
        width=alt.Step(step),
    )
)
text3 = bars3.mark_text(color=colors["eco-mid-blue"], yOffset=-7, xOffset=0).encode(
    text=alt.Text("value:N", format=".0f")
)
text3b = (
    text3.mark_text(color=colors["eco-mid-blue"], angle=270, align="left", yOffset=-3)
    .encode(text="figure:N", x="x:N", y="y:Q")
    .transform_calculate(x="' '", y="0")
    .transform_filter("datum.value<15")
)
bars4 = (
    base.mark_bar(color=colors["eco-blue"], opacity=0.8)
    .transform_filter("datum.figure=='Ethnicity'")
    .properties(
        height=height,
        width=alt.Step(step),
    )
)
text4 = bars4.mark_text(color=colors["eco-blue"], yOffset=-7, xOffset=0).encode(
    text=alt.Text("value:N", format=".0f")
)
text4b = (
    text4.mark_text(color=colors["eco-blue"], angle=270, align="left", yOffset=-3)
    .encode(text="figure:N", x="x:N", y="y:Q")
    .transform_calculate(x="' '", y="0")
    .transform_filter("datum.value<6")
)
layer = alt.hconcat(
    bars1 + text1 + text1b,
    bars2 + text2 + text2b,
    bars3 + text3 + text3b,
    bars4 + text4 + text4b,
).configure_view(stroke=None)
layer.save("visualisation/" + f + ".json")
layer

alt.HConcatChart(...)

# Fig 8

In [75]:
df = pd.read_excel(
    "raw/Data for graphs (Rodriguez & Ifan)  ECO3.xlsx",
    sheet_name="Figure 8",
    skiprows=1,
    usecols="A:C",
).dropna(how="all")

In [76]:
f = "fig8_wfh"
f8 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f8.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f8 = df
df.head()

,ca_blwah,region_n,percentage_homeworking
0,Always,North East,5.277778
1,Always,North West,6.316813
2,Always,Yorkshire,4.859611
3,Always,East Midlands,5.776173
4,Always,West Midlands,5.990783


In [77]:
bars = (
    alt.Chart(f8)
    .mark_bar(height=16, opacity=0.8)
    .encode(
        color=alt.Color(
            "ca_blwah:N",
            legend=alt.Legend(title="", padding=1),
            scale=alt.Scale(
                domain=df["ca_blwah"].unique(),
                range=[
                    colors["eco-turquiose"],
                    colors["eco-light-blue"],
                    colors["eco-mid-blue"],
                ],
            ),
        ),
        x=alt.X(
            "percentage_homeworking:Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="% of workers",
                format=".0f",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleAnchor="end",
                titleY=20,
                titleFontWeight="normal",
                titleFontSize=10,
            ),
            scale=alt.Scale(domain=[0, 40]),
        ),
        order="ca_blwah:N",
        y=alt.Y(
            "region_n:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
    )
)
text = bars.mark_text(dx=-5, dy=1, align="right").encode(
    text=alt.Text("percentage_homeworking:Q", format=".0f"),
    color=alt.condition(
        datum.SAmmount < 8,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
    x=alt.X(
        "percentage_homeworking:Q",
        stack="zero",
    ),
)
layer1 = (
    ((bars + text).properties(height=300, width=400))
    .configure_legend(
        titleColor=colors["eco-gray"], labelColor=colors["eco-gray"], offset=-40
    )
    .configure_view(stroke=None)
)
layer1.save("visualisation/" + f + ".json")
layer1

alt.LayerChart(...)